<a href="https://colab.research.google.com/github/alinecrb/n1-cabrini/blob/main/N1_Cabrini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import requests
from datetime import datetime
import pytz

# Constants
IP_ADDRESS = "3.92.0.131"
PORT_STH = 8666
DASH_HOST = "0.0.0.0"

def get_luminosity_data(lastN):
    url = f"http://{IP_ADDRESS}:{PORT_STH}/STH/v1/contextEntities/type/Lamp/id/urn:ngsi-ld:Lamp:001/attributes/luminosity?lastN={lastN}"
    headers = {'fiware-service': 'smart','fiware-servicepath': '/'}
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        try:
            return r.json()['contextResponses'][0]['contextElement']['attributes'][0]['values']
        except KeyError:
            return []
    else:
        print(f"Error {r.status_code} on luminosity")
        return []

def get_humidity_data(lastN):
    url = f"http://{IP_ADDRESS}:{PORT_STH}/STH/v1/contextEntities/type/Dht/id/urn:ngsi-ld:Dht:001/attributes/humidity?lastN={lastN}"
    headers = {'fiware-service': 'smart','fiware-servicepath': '/'}
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        try:
            return r.json()['contextResponses'][0]['contextElement']['attributes'][0]['values']
        except KeyError:
            return []
    else:
        print(f"Error {r.status_code} on humidity")
        return []

def get_temperature_data(lastN):
    url = f"http://{IP_ADDRESS}:{PORT_STH}/STH/v1/contextEntities/type/Dht/id/urn:ngsi-ld:Dht:001/attributes/temperature?lastN={lastN}"
    headers = {'fiware-service': 'smart','fiware-servicepath': '/'}
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        try:
            return r.json()['contextResponses'][0]['contextElement']['attributes'][0]['values']
        except KeyError:
            return []
    else:
        print(f"Error {r.status_code} on temperature")
        return []

def convert_to_lisbon_time(timestamps):
    utc = pytz.utc
    lisbon = pytz.timezone('Europe/Lisbon')
    converted = []
    for ts in timestamps:
        ts = ts.replace('T',' ').replace('Z','')
        try:
            dt = datetime.strptime(ts, '%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            dt = datetime.strptime(ts, '%Y-%m-%d %H:%M:%S')
        converted.append(utc.localize(dt).astimezone(lisbon))
    return converted

lastN = 10  # últimos 10 pontos

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Sensor Data Viewer (Luminosidade, Umidade, Temperatura)'),

    html.Div([
        dcc.Graph(id='graph-luminosity'),
        dcc.Graph(id='graph-humidity'),
        dcc.Graph(id='graph-temperature'),
    ]),

    dcc.Store(
        id='sensor-data-store',
        data={
            'timestamps_luminosity': [],
            'luminosity_values': [],
            'timestamps_humidity': [],
            'humidity_values': [],
            'timestamps_temperature': [],
            'temperature_values': []
        }
    ),

    dcc.Interval(
        id='interval-component',
        interval=10 * 1000,  # 10 segundos
        n_intervals=0
    )
])

@app.callback(
    Output('sensor-data-store', 'data'),
    Input('interval-component', 'n_intervals'),
    State('sensor-data-store', 'data')
)
def update_data_store(n, stored):
    # Luminosidade
    lum = get_luminosity_data(lastN)
    if lum:
        ts_l = [e['recvTime'] for e in lum]
        vs_l = [float(e['attrValue']) for e in lum]
        ts_l = convert_to_lisbon_time(ts_l)
        stored['timestamps_luminosity'] = ts_l
        stored['luminosity_values'] = vs_l

    # Umidade
    hum = get_humidity_data(lastN)
    if hum:
        ts_h = [e['recvTime'] for e in hum]
        vs_h = [float(e['attrValue']) for e in hum]
        ts_h = convert_to_lisbon_time(ts_h)
        stored['timestamps_humidity'] = ts_h
        stored['humidity_values'] = vs_h

    # Temperatura
    temp = get_temperature_data(lastN)
    if temp:
        ts_t = [e['recvTime'] for e in temp]
        vs_t = [float(e['attrValue']) for e in temp]
        ts_t = convert_to_lisbon_time(ts_t)
        stored['timestamps_temperature'] = ts_t
        stored['temperature_values'] = vs_t

    return stored

@app.callback(
    Output('graph-luminosity', 'figure'),
    Output('graph-humidity', 'figure'),
    Output('graph-temperature', 'figure'),
    Input('sensor-data-store', 'data')
)
def update_graphs(data):
    fig_lum = go.Figure()
    fig_hum = go.Figure()
    fig_temp = go.Figure()

    if data['timestamps_luminosity']:
        fig_lum.add_trace(go.Scatter(
            x=data['timestamps_luminosity'],
            y=data['luminosity_values'],
            mode='lines+markers',
            name='Luminosidade'
        ))
        fig_lum.update_layout(title='Luminosidade ao Longo do Tempo', xaxis_title='Tempo', yaxis_title='Luminosidade')

    if data['timestamps_humidity']:
        fig_hum.add_trace(go.Scatter(
            x=data['timestamps_humidity'],
            y=data['humidity_values'],
            mode='lines+markers',
            name='Umidade'
        ))
        fig_hum.update_layout(title='Umidade ao Longo do Tempo', xaxis_title='Tempo', yaxis_title='Umidade (%)')

    if data['timestamps_temperature']:
        fig_temp.add_trace(go.Scatter(
            x=data['timestamps_temperature'],
            y=data['temperature_values'],
            mode='lines+markers',
            name='Temperatura'
        ))
        fig_temp.update_layout(title='Temperatura ao Longo do Tempo', xaxis_title='Tempo', yaxis_title='Temperatura (°C)')

    return fig_lum, fig_hum, fig_temp

if __name__ == '__main__':
    app.run(debug=True, host=DASH_HOST, port=8050)

<IPython.core.display.Javascript object>